coding=utf-8
Copyright 2024 The Google Research Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

"""X-MAGICAL cross-embodiment pretraining script."""


In [1]:

import os.path as osp
import subprocess
import os
from configs.constants import XMAGICAL_EMBODIMENT_TO_ENV_NAME

from absl import app
from absl import logging
from configs.constants import ALGORITHMS
from configs.constants import EMBODIMENTS
from torchkit.experiment import string_from_kwargs
from torchkit.experiment import unique_id
import yaml
import random


2024-11-27 20:43:06.371845: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 20:43:06.371917: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 20:43:06.371936: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 20:43:06.382533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# pylint: disable=logging-fstring-interpolation

# Mapping from pretraining algorithm to config file.
ALGO_TO_CONFIG = {
    "xirl": "configs/xmagical/pretraining/tcc.py",
    "lifs": "configs/xmagical/pretraining/lifs.py",
    "tcn": "configs/xmagical/pretraining/tcn.py",
    "goal_classifier": "configs/xmagical/pretraining/classifier.py",
    "raw_imagenet": "configs/xmagical/pretraining/imagenet.py",
}


In [3]:
# We want to pretrain on the entire 1k demonstrations.
MAX_DEMONSTRATIONS = -1
# embodiment = None
embodiment = "longstick"
algo = "xirl"
unique_name = True,
random_number = random.randint(1, 1000)  # You can adjust the range as needed
experiment_name = f"/home/user/xirl/exp/exp{random_number}"
experiment_name = "/home/user/xirl/exp/same_long"

# device = 'cpu'
device = 'cuda:0'
embodiments = EMBODIMENTS if embodiment is None else [embodiment]
print("experiment name: ",experiment_name)

experiment name:  /home/user/xirl/exp/same_long


In [4]:

for embodiment in embodiments:
    # Generate a unique experiment name.
    print("embodiment: ",embodiment)
    kwargs = {
        "dataset": "xmagical",
        "mode": "cross",
        "algo": algo,
        "embodiment": embodiment,
    }
    if unique_name:
      kwargs["uid"] = unique_id()
    logging.info("Experiment name: %s", experiment_name)
    
    # Train on all classes but the given embodiment.
    trainable_embs = tuple(EMBODIMENTS - set([embodiment]))
    try:
        print("Start of experment\n")
        subprocess.run(
            [
                "python",
                "pretrain.py",
                "--device",
                device,
                "--experiment_name",
                experiment_name,
                "--raw_imagenet" if algo == "raw_imagenet" else " ",
                "--config",
                f"{ALGO_TO_CONFIG[algo]}",
                "--config.data.pretrain_action_class",
                f"{repr(trainable_embs)}",
                "--config.data.downstream_action_class",
                f"{repr(trainable_embs)}",
                "--config.data.max_vids_per_class",
                f"{MAX_DEMONSTRATIONS}",
            ],
            check=True,
            # stdout=subprocess.PIPE,  # Capture standard output
            # stderr=subprocess.PIPE,   # Capture standard error
            text=True  
        )
        

        # Note: This assumes that the config.root_dir value has not been
        # changed to its default value of 'tmp/xirl/pretrain_runs/'.
        exp_path = osp.join("/tmp/xirl/pretrain_runs/", experiment_name)
        # print("Output:", result.stdout)
        print("end of experment\n")
    except subprocess.CalledProcessError as e:
        print("Error:", e.stderr)
        print("Return Code:", e.returncode)


embodiment:  longstick
Start of experment



2024-11-27 20:43:12.889676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 20:43:12.889745: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 20:43:12.889764: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 20:43:12.899530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 20:43:18.339738: I te

Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]
Error: None
Return Code: 1


In [5]:
for embodiment in embodiments:
    # The 'goal_classifier' baseline does not need to compute a goal embedding.
    if algo != "goal_classifier":
      subprocess.run(
          [
              "python",
              "compute_goal_embedding.py",
              "--experiment_path",
              experiment_name,
          ],
          check=True,
      )


2024-11-27 20:43:23.664565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 20:43:23.664631: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 20:43:23.664650: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 20:43:23.674244: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 20:43:29.029873: I te

Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]


Traceback (most recent call last):
  File "/home/user/xirl/compute_goal_embedding.py", line 96, in <module>
    app.run(main)
  File "/home/user/mambaforge/envs/env/lib/python3.11/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/home/user/mambaforge/envs/env/lib/python3.11/site-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
             ^^^^^^^^^^
  File "/home/user/xirl/compute_goal_embedding.py", line 81, in main
    model, downstream_loader = setup()
                               ^^^^^^^
  File "/home/user/xirl/compute_goal_embedding.py", line 65, in setup
    downstream_loaders = common.get_downstream_dataloaders(config, False)["train"]
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/xirl/xirl/common.py", line 95, in get_downstream_dataloaders
    "train": _loader("train"),
             ^^^^^^^^^^^^^^^^
  File "/home/user/xirl/xirl/common.py", line 80, in _loader
    datasets = fact

CalledProcessError: Command '['python', 'compute_goal_embedding.py', '--experiment_path', '/home/user/xirl/exp/same_long']' returned non-zero exit status 1.

In [ ]:
graph_data_path = f"./data/{embodiment}/graphs/combined_graph.pt"

for embodiment in embodiments:
    # The 'goal_classifier' baseline does not need to compute a goal embedding.
    if algo != "goal_classifier":
      subprocess.run(
          [
              "python",
              "compute_goal_embedding.py",
              "--experiment_path",
              experiment_name,
              "--graph_data_path",
              graph_data_path
          ],
          check=True,
      )
    
    # Dump experiment metadata as yaml file.
    # with open(osp.join(experiment_name, "metadata.yaml"), "w") as fp:
    #   yaml.dump(kwargs, fp)


In [ ]:
import utils

self_supervised_emb = utils.load_pickle(experiment_name, "self_supervised_emb.pkl")
self_supervised_distance_scale = utils.load_pickle(experiment_name, "self_supervised_distance_scale.pkl")
# gnn_emb = utils.load_pickle(experiment_name, "gnn_emb.pkl")
# gnn_distance_scale = utils.load_pickle(experiment_name, "gnn_distance_scale.pkl")
print(self_supervised_emb, self_supervised_distance_scale)


In [ ]:
print(self_supervised_emb)


In [ ]:
print(gnn_emb)